In [11]:
# %conda install cudatoolkit
!nvidia-smi


Thu May  2 15:57:06 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    Off | 00000000:00:1E.0 Off |                    0 |
|  0%   40C    P0              61W / 300W |   4640MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [1]:
%pip install --upgrade torch

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-qoiy_lnm
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-qoiy_lnm
  Resolved https://github.com/huggingface/transformers to commit 39359e5b5f6246c667cc25f743437cb05d492ea7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Note: you may need to restart the kernel to use updated packages.


In [ ]:
print(torch.__version__)
print(torch.cuda.is_available())

In [3]:
import pandas as pd

In [ ]:
!pip install pipreqs
!pipreqs /home/sagemaker-user/exploration/MISTRAL_JOSE

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.3/798.3 kB 55.7 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=84d0b5486c2d2c53e5cc63bfcd8b14f5912c2d1db5bd8d5e9aeeb72e5759c483
  Stored in directory: /home/sagemaker-user/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt
  Attempting uninstall: ipython
    Found existing installation: ipython 8.22.2
    Uninstalling ipython-8.22.2:
      Successfully uninstalled ipython-8.22.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
hdijupyterutils 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.4 which is incompatible.
sparkmagic 0.21.0 requires pandas<2.0.0,>=0.17.1, but you have pandas 2.1.4 which is incompatible.


In [5]:
!pigar generate

Requirements file has been overwritten, no difference.
Requirements has been written to /home/sagemaker-user/exploration/MISTRAL_JOSE/requirements.txt.


In [3]:
!pipreqs /home/sagemaker-user/exploration/MISTRAL_JOSE/

INFO: Not scanning for jupyter notebooks.


### Load Data

In [4]:
test_df = pd.read_parquet("s3://cleo-data-science/mlflow/40/4208e04c9f0e442bbff73bd6996bcf8b/artifacts/input_data/test.parquet")

In [5]:
test_df.head()

,Unnamed: 0.1,Unnamed: 0,index,bot_request_id,bot_request_created_date,first_annotation_date,classifier_label,num_annotations,company_annotated,max_annotations_per_intent,max_agreement,agreed_label,normalised_bot_request_body,prompt,completion
0,18134,18134,18134,223796853,2023-05-13,2023-05-20,send_overdraft_protection,4,cleo,3,0.750000,send_overdraft_protection,Can I get a advance,USER: Can I get a advance\n\n###\n\n,86
1,9123,9123,9123,223118006,2023-05-01,2023-05-11,send_overdraft_protection,4,cleo,4,1.000000,send_overdraft_protection,Loan,USER: home\nCLEO: unknown\nUSER: home\nCLEO: u...,86
2,18151,18151,18151,223797771,2023-05-13,2023-05-15,send_add_account,4,cleo,4,1.000000,atm,Cleo affiliated atm,USER: Can I use Cleo card to withdraw funds at...,1
3,17639,17639,17639,223767952,2023-05-12,2023-05-15,send_faq,3,cleo,2,0.666667,send_help,Help,"USER: Debit card\nCLEO: No problem, Rochelle. ...",66
4,9919,9919,9919,223183772,2023-05-02,2023-05-11,send_unsubscribe_from_premium,4,cleo,4,1.000000,send_unsubscribe_from_premium,Unsubscribe now please,USER: I would like to speak to an actual perso...,114


In [6]:
import os

import numpy as np
import pandas as pd
import torch
from peft import PeftModel
from transformers import (
    AutoTokenizer,
    BitsAndBytesConfig,
    MistralForSequenceClassification,
)

2024-05-02 15:50:16.012254: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
%pip install bitsandbytes

Note: you may need to restart the kernel to use updated packages.


In [8]:
base_model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.bfloat16,
        low_cpu_mem_usage=True
)   
base_model = MistralForSequenceClassification.from_pretrained(
        base_model_id,
        num_labels=128,
        quantization_config=bnb_config,
    )

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at mistralai/Mistral-7B-v0.1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
